In [1]:
!pip install git+https://github.com/rongardF/tvdatafeed  backtesting
!pip install tradingview-screener==2.5.0
!pip install openpyxl
!pip install statsmodels
!pip install tqdm
!pip install tradingview-ta

import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols

# EMA hesaplama fonksiyonu
def ema(src, span):
    return src.ewm(span=span, adjust=False).mean()

# Normal RSI hesaplama fonksiyonu (RMA kullanarak)
def calculate_rsi(data, period=14):
    delta = data.diff()  # Fiyat değişimini hesapla
    gain = delta.where(delta > 0, 0)  # Kazançları filtrele
    loss = -delta.where(delta < 0, 0)  # Kayıpları filtrele

    avg_gain = gain.rolling(window=period, min_periods=1).mean()  # Kazançların ortalaması
    avg_loss = loss.rolling(window=period, min_periods=1).mean()  # Kayıpların ortalaması

    rs = avg_gain / avg_loss  # Kazanç kayıp oranı
    rsi = 100 - (100 / (1 + rs))  # RSI hesaplama
    return rsi


# Göreceli RSI hesaplama fonksiyonu
def calculate_relative_rsi(data, rsi_period=14, relative_rsi_period=14):
    # Normal RSI hesapla
    rsi = calculate_rsi(data, rsi_period)

    # RSI'nın hareketli ortalamasını al
    relative_rsi = rsi.rolling(window=relative_rsi_period, min_periods=1).mean()
    return relative_rsi

# Zaman dilimi seçimi fonksiyonu
def select_interval():
    print("\nLütfen zaman dilimini seçin:")
    print("1: Günlük")
    print("2: Haftalık")
    print("3: Aylık")
    choice = input("Seçiminiz (1/2/3): ")

    interval_map = {
        "1": Interval.in_daily,    # Günlük
        "2": Interval.in_weekly,   # Haftalık
        "3": Interval.in_monthly   # Aylık
    }

    return interval_map.get(choice, Interval.in_daily)  # Varsayılan olarak günlük interval seçilsin

# TradingView bağlantısı
tv = TvDatafeed()

# Zaman dilimini kullanıcıdan alıyoruz
selected_interval = select_interval()

# Hisseler listesini otomatik çekmek için tradingview-screener kullanıyoruz
Hisseler = get_all_symbols(market='turkey')  # Türkiye borsasındaki tüm hisseleri al

# Sonuçları saklamak için DataFrame
columns = ['Hisse', 'Fiyat', 'Fiyat/XU100', 'Normal_RSI', 'Göreceli_RSI',
           'EMA233_Durumu', 'EMA144_Durumu', 'EMA89_Durumu', 'EMA34_Durumu', 'Üstünde']
df_results = pd.DataFrame(columns=columns)

# Toplam hisse sayısını öğreniyoruz
total_hisseler = len(Hisseler)

for index, hisse in enumerate(Hisseler, start=1):
    try:
        # Seçilen zaman dilimine göre hisse ve XU100 verilerini alma
        data = tv.get_hist(symbol=hisse, exchange='BIST', interval=selected_interval, n_bars=300)
        dataref = tv.get_hist(symbol='XU100', exchange='BIST', interval=selected_interval, n_bars=300)

        # Verileri resetliyoruz (index sıfırlama)
        data = data.reset_index()
        dataref = dataref.reset_index()

        # Fiyatları XU100 endeksine göre normalize etme
        result = data.copy()
        columns_to_divide = ['open', 'high', 'low', 'close', 'volume']  r
        for column in columns_to_divide:
            result[column] = 100 * (data[column] / dataref[column])

        # EMA hesaplama
        result['EMA_233'] = ema(result['close'], 233)
        result['EMA_144'] = ema(result['close'], 144)
        result['EMA_89'] = ema(result['close'], 89)
        result['EMA_34'] = ema(result['close'], 34)

        # Normal RSI hesaplama
        result['Normal_RSI'] = calculate_rsi(result['close'])

        # Göreceli RSI hesaplama
        result['Göreceli_RSI'] = calculate_relative_rsi(result['close'])

        # EMA durum kontrolü
        latest = result.iloc[-1]
        ema_durumları = {
            'EMA233_Durumu': 'Üstünde' if latest['close'] > latest['EMA_233'] else 'Altında',
            'EMA144_Durumu': 'Üstünde' if latest['close'] > latest['EMA_144'] else 'Altında',
            'EMA89_Durumu': 'Üstünde' if latest['close'] > latest['EMA_89'] else 'Altında',
            'EMA34_Durumu': 'Üstünde' if latest['close'] > latest['EMA_34'] else 'Altında'
        }

        # "Üstünde" toplam sayısı
        üstünde_sayısı = sum([1 for durum in ema_durumları.values() if durum == 'Üstünde'])

        # Satır ekleme
        row = {
            'Hisse': hisse,
            'Fiyat': round(latest['close'], 2),
            'Fiyat/XU100': round(latest['close'] / dataref['close'].iloc[-1], 6),  # XU100 ile normalize edilmiş fiyat
            'Normal_RSI': round(latest['Normal_RSI'], 2),
            'Göreceli_RSI': round(latest['Göreceli_RSI'], 2),  # Göreceli RSI
            **ema_durumları,
            'Üstünde': üstünde_sayısı
        }

        # Sonuçları DataFrame'e ekle
        df_results = pd.concat([df_results, pd.DataFrame([row])], ignore_index=True)

        # İlerlemeyi sayısal olarak göster
        print(f"İşlenen Hisse: {hisse} ({index}/{total_hisseler})")

    except Exception as e:
        print(f"{hisse} için veri alınamadı: {e}")

# Tabloyu Excel'e kaydetme
df_results.to_excel("hisse_analiz_tablosu.xlsx", index=False)
print("Tablo başarıyla kaydedildi: hisse_analiz_tablosu.xlsx")

# Tarama tamamlandığında tabloyu ekrana yazdırma
print("\nTarama tamamlandı! Sonuçlar:")
print(df_results)


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-blgrsf_e
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-blgrsf_e
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=f1c03c35d5c956a9f3effacb5aef5bbdbe1943a61f4796faa080fb5d5b587805
  Stored in directory: /tmp/pip-ephem-wheel-cache-l63t53_6/wheels/e4/32/1e/21ebcacc6549d75fae3bf3ff75cee8fcbe4e5c189d88fbd5a1
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173912 sha256=42b9f091e656f305a28ed38e9f086ee712c6179e7a8f907428696bffad908b1b
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f51034324906


Lütfen zaman dilimini seçin:
1: Günlük
2: Haftalık
3: Aylık
Seçiminiz (1/2/3): 1


<ipython-input-1-99c49fd1984e>:124: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, pd.DataFrame([row])], ignore_index=True)


İşlenen Hisse: BIST:VANGD (1/583)
İşlenen Hisse: BIST:SNPAM (2/583)
İşlenen Hisse: BIST:EKOS (3/583)
İşlenen Hisse: BIST:PKENT (4/583)
İşlenen Hisse: BIST:PRZMA (5/583)
İşlenen Hisse: BIST:VAKBN (6/583)
İşlenen Hisse: BIST:CONSE (7/583)
İşlenen Hisse: BIST:PENTA (8/583)
İşlenen Hisse: BIST:QNBTR (9/583)
İşlenen Hisse: BIST:GLBMD (10/583)
İşlenen Hisse: BIST:KENT (11/583)
İşlenen Hisse: BIST:KGYO (12/583)
İşlenen Hisse: BIST:BORSK (13/583)
İşlenen Hisse: BIST:FMIZP (14/583)
İşlenen Hisse: BIST:BALAT (15/583)
İşlenen Hisse: BIST:AKSA (16/583)
İşlenen Hisse: BIST:SUNTK (17/583)
İşlenen Hisse: BIST:ALVES (18/583)
İşlenen Hisse: BIST:IMASM (19/583)
İşlenen Hisse: BIST:YAPRK (20/583)
İşlenen Hisse: BIST:YBTAS (21/583)
İşlenen Hisse: BIST:OTKAR (22/583)
İşlenen Hisse: BIST:ALFAS (23/583)
İşlenen Hisse: BIST:SASA (24/583)
İşlenen Hisse: BIST:Z30KP (25/583)
İşlenen Hisse: BIST:EUHOL (26/583)
İşlenen Hisse: BIST:TMSN (27/583)
İşlenen Hisse: BIST:PRDGS (28/583)
İşlenen Hisse: BIST:MNDRS (29/583)


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


BIST:USAK için veri alınamadı: 'NoneType' object has no attribute 'reset_index'
İşlenen Hisse: BIST:BAYRK (372/583)
İşlenen Hisse: BIST:GLRYH (373/583)
İşlenen Hisse: BIST:KRDMD (374/583)
İşlenen Hisse: BIST:PETUN (375/583)
İşlenen Hisse: BIST:AKBNK (376/583)
İşlenen Hisse: BIST:AKMGY (377/583)
İşlenen Hisse: BIST:KLSER (378/583)
İşlenen Hisse: BIST:SAYAS (379/583)
İşlenen Hisse: BIST:DIRIT (380/583)
İşlenen Hisse: BIST:SILVR (381/583)
İşlenen Hisse: BIST:EUKYO (382/583)
İşlenen Hisse: BIST:GARAN (383/583)
İşlenen Hisse: BIST:CGCAM (384/583)
İşlenen Hisse: BIST:MSGYO (385/583)
İşlenen Hisse: BIST:YGGYO (386/583)
İşlenen Hisse: BIST:ISSEN (387/583)
İşlenen Hisse: BIST:ISFIN (388/583)
İşlenen Hisse: BIST:KLMSN (389/583)
İşlenen Hisse: BIST:CEMAS (390/583)
İşlenen Hisse: BIST:IZFAS (391/583)
İşlenen Hisse: BIST:TKNSA (392/583)
İşlenen Hisse: BIST:BTCIM (393/583)
İşlenen Hisse: BIST:DERHL (394/583)
İşlenen Hisse: BIST:A1CAP (395/583)
İşlenen Hisse: BIST:KARTN (396/583)
İşlenen Hisse: BIST:

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


BIST:PETKM için veri alınamadı: 'NoneType' object has no attribute 'reset_index'
İşlenen Hisse: BIST:ARENA (469/583)
İşlenen Hisse: BIST:ORMA (470/583)
İşlenen Hisse: BIST:OSMEN (471/583)
İşlenen Hisse: BIST:FADE (472/583)
İşlenen Hisse: BIST:ATEKS (473/583)
İşlenen Hisse: BIST:EGEEN (474/583)
İşlenen Hisse: BIST:LILAK (475/583)
İşlenen Hisse: BIST:PRKAB (476/583)
İşlenen Hisse: BIST:PATEK (477/583)
İşlenen Hisse: BIST:CANTE (478/583)
İşlenen Hisse: BIST:MPARK (479/583)
İşlenen Hisse: BIST:LINK (480/583)
İşlenen Hisse: BIST:SEGMN (481/583)
İşlenen Hisse: BIST:SEKFK (482/583)
İşlenen Hisse: BIST:OZGYO (483/583)
İşlenen Hisse: BIST:GOZDE (484/583)
İşlenen Hisse: BIST:KAYSE (485/583)
İşlenen Hisse: BIST:DNISI (486/583)
İşlenen Hisse: BIST:CCOLA (487/583)
İşlenen Hisse: BIST:PGSUS (488/583)
İşlenen Hisse: BIST:DURKN (489/583)
İşlenen Hisse: BIST:ALMAD (490/583)
İşlenen Hisse: BIST:AVPGY (491/583)
İşlenen Hisse: BIST:KAREL (492/583)
İşlenen Hisse: BIST:LMKDC (493/583)
İşlenen Hisse: BIST:MA

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


BIST:GLDTR için veri alınamadı: 'NoneType' object has no attribute 'reset_index'
İşlenen Hisse: BIST:BJKAS (546/583)
İşlenen Hisse: BIST:SRVGY (547/583)
İşlenen Hisse: BIST:FROTO (548/583)
İşlenen Hisse: BIST:MTRYO (549/583)
İşlenen Hisse: BIST:DZGYO (550/583)
İşlenen Hisse: BIST:IHLGM (551/583)
İşlenen Hisse: BIST:ISCTR (552/583)
İşlenen Hisse: BIST:EREGL (553/583)
İşlenen Hisse: BIST:HALKB (554/583)
İşlenen Hisse: BIST:TTRAK (555/583)
İşlenen Hisse: BIST:DERIM (556/583)
İşlenen Hisse: BIST:AGHOL (557/583)
İşlenen Hisse: BIST:VAKFN (558/583)
İşlenen Hisse: BIST:TSKB (559/583)
İşlenen Hisse: BIST:OZRDN (560/583)
İşlenen Hisse: BIST:ALARK (561/583)
İşlenen Hisse: BIST:DARDL (562/583)
İşlenen Hisse: BIST:RNPOL (563/583)
İşlenen Hisse: BIST:BIMAS (564/583)
İşlenen Hisse: BIST:GRNYO (565/583)
İşlenen Hisse: BIST:GARFA (566/583)
İşlenen Hisse: BIST:EDATA (567/583)
İşlenen Hisse: BIST:ORCAY (568/583)
İşlenen Hisse: BIST:GUNDG (569/583)
İşlenen Hisse: BIST:ZPLIB (570/583)
İşlenen Hisse: BIST: